# arUSD trading volume by wallet

## How to start
### Get an APPI key
You need to register an account with [Bitquery](https://graphql.bitquery.io/ide). You get your personal API key which you need to run this script.

# Run the notebook
Click on ![](icons/d_play.png) and confirm the popup asking about a kernel restart and provide the Bitquery API key when asked. In a few seconds you will get the result.

Alternatively you can run this notebook locally and create a `.env` file whih contains the key. It will be automatically loaded. See the file `.env.example` for an example. 

## About the data returned
It is assumed that the address swapping "in" is the same that is swapping "out". Synth-to-synth swaps and transfers of synths between wallets are not considered.

arUSD contract: 0x025AB35fF6AbccA56d57475249baaEae08419039

Swapping contract: 0x948011e8ca8df1e9c83fee88967a5fc30c7a604b

Fee holder contract: 0xc768915863a333db9bb871bd687dd0a0ae41a3be

In [ ]:
import os
from pathlib import Path
#import scripts.utils as utils
import utils
import pandas as pd
#from IPython.display import display, HTML
import matplotlib.pyplot as plt
#%matplotlib inline
from IPython.display import HTML
# pd.option_context('display.precision', 3)
# pd.options.display.float_format = '{:,.5e}'.format
api_key = None
if not os.path.isfile('.env'):
	api_key = input("Please enter your Bitquery API key. You can get one at https://graphql.bitquery.io/ide and paste it here: ")

start_date = "2022-06-01"

query_variables = {"start_date": start_date}

# Get all swap-out transactions
query = """
query ($start_date: ISO8601DateTime) {
ethereum(network: avalanche) {
    transfers(
			options: {
				limit:100000
				desc: ["amount"]
			}
			success: true
			date: {since: $start_date}
			receiver:{notIn:["0xc768915863a333db9bb871bd687dd0a0ae41a3be","0x948011e8ca8df1e9c83fee88967a5fc30c7a604b"]}
			sender:{is:"0x0000000000000000000000000000000000000000"}
			currency: {is: "0x025AB35fF6AbccA56d57475249baaEae08419039"}
		) {
		receiver {address}
		amount
		currency {symbol, address}
   }
  }
}
"""

swap_out_result = utils.bitqueryAPICall(query, query_variables, api_key)

swap_out = []
for r in swap_out_result["data"]["ethereum"]["transfers"]:
    swap_out.append([r["receiver"]["address"], r["amount"]])

# Get all swap-in transactions
query = """
query ($start_date: ISO8601DateTime) {
ethereum(network: avalanche) {
    transfers(
			options: {
				limit:100000
				desc: ["amount"]
			}
			success: true
			date: {since: $start_date}
			receiver:{is:"0x948011e8ca8df1e9c83fee88967a5fc30c7a604b"}
			sender:{notIn:["0x0000000000000000000000000000000000000000", "0xc768915863a333db9bb871bd687dd0a0ae41a3be"]}
			currency: {is: "0x025AB35fF6AbccA56d57475249baaEae08419039"}
		) {
		sender {address}
		amount
		currency {symbol, address}
   }
  }
}
"""

swap_in_result = utils.bitqueryAPICall(query, query_variables, api_key)

swap_in = []
for r in swap_in_result["data"]["ethereum"]["transfers"]:
    swap_in.append([r["sender"]["address"], r["amount"]])

# Get last transaction date
query = """
query ($start_date: ISO8601DateTime) {
ethereum(network: avalanche) {
    transfers(
			options: {
				limit:1
				desc: ["block.timestamp.iso8601"]
			}
			success: true
			date: {since: $start_date}
			receiver:{notIn:["0xc768915863a333db9bb871bd687dd0a0ae41a3be","0x948011e8ca8df1e9c83fee88967a5fc30c7a604b"]}
			sender:{is:"0x0000000000000000000000000000000000000000"}
			currency: {is: "0x025AB35fF6AbccA56d57475249baaEae08419039"}
		) {
		block {timestamp {iso8601}}
   }
  }
}
"""
last_out = utils.bitqueryAPICall(query, query_variables, api_key)

query = """
query ($start_date: ISO8601DateTime) {
ethereum(network: avalanche) {
    transfers(
			options: {
				limit:1
				desc: ["block.timestamp.iso8601"]
			}
			success: true
			date: {since: $start_date}
			receiver:{is:"0x948011e8ca8df1e9c83fee88967a5fc30c7a604b"}
			sender:{notIn:["0x0000000000000000000000000000000000000000", "0xc768915863a333db9bb871bd687dd0a0ae41a3be"]}
			currency: {is: "0x025AB35fF6AbccA56d57475249baaEae08419039"}
		) {
		block {timestamp {iso8601}}
   }
  }
}
"""
last_in = utils.bitqueryAPICall(query, query_variables, api_key)
last_tx_date=max(last_in["data"]["ethereum"]["transfers"][0]["block"]["timestamp"]["iso8601"],last_out["data"]["ethereum"]["transfers"][0]["block"]["timestamp"]["iso8601"])

# Filter out arUSD swap-out transactions from wallets that never performed a swap-in
df_out = utils.df(swap_out, columns=["address", "amount_out"])
df_in = utils.df(swap_in, columns=["address", "amount_in"])
int_df = pd.merge(df_out, df_in, how='inner', on=['address'])

# sum of amount_in and amount_out
int_df['amount'] = int_df['amount_in'] + int_df['amount_out']

# Output
print("Most recent transaction: "+last_tx_date)

# Sum of all swap-in and swap-out transactions
print("Total volume: "+str(int_df['amount'].sum().round(2)))


int_df.round(2)

# display(int_df)
